This code is adapted based on code examples in Ramsundar, Bharath; Eastman, Peter; Walters, Patrick; Pande, Vijay. Deep Learning for the Life Sciences, Chapter 7.

# Installing DeepChem

In [0]:
# Installing RDKit
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

In [0]:
# append rdkit path to current python system path.
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [0]:
# Install DeepChem 
!pip install deepchem

In [0]:
import deepchem as dc
import deepchem.models.tensorgraph.layers as layers
import numpy as np
import os
import re

# Dataset
To run this example, you will need to download the Broad BBBC005 dataset from https://data.broadinstitute.org/bbbc/BBBC005/. No login or registration is needed to download this dataset, so the raw images can simply be fetched with

! wget https://data.broadinstitute.org/bbbc/BBBC005/BBBC005_v1_images.zip unzip BBBC005_v1_images.zip

The ground-truth segmentation masks can be fetched as follows

! wget https://data.broadinstitute.org/bbbc/BBBC005/BBBC005_v1_ground_truth.zip unzip BBBC005_v1_ground_truth.zip

In [0]:
# The ground-truth segmentation masks can be fetched as follows

! wget https://data.broadinstitute.org/bbbc/BBBC005/BBBC005_v1_ground_truth.zip

In [0]:
! unzip BBBC005_v1_ground_truth.zip

In [0]:
# check the files on colab drive
! ls

In [0]:
# Load the datasets.
image_dir = 'BBBC005_v1_images'
label_dir = 'BBBC005_v1_ground_truth'
rows = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P')
blurs = (1, 4, 7, 10, 14, 17, 20, 23, 26, 29, 32, 35, 39, 42, 45, 48)
files = []
labels = []
for f in os.listdir(label_dir):
  if f.endswith('.TIF'):
    for row, blur in zip(rows, blurs):
      fname = f.replace('_F1', '_F%d'%blur).replace('_A', '_%s'%row)
      files.append(os.path.join(image_dir, fname))
      labels.append(os.path.join(label_dir, f))

loader = dc.data.ImageLoader()
dataset = loader.featurize(files, labels)
splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset, seed=123)

In [0]:
# inspect the sets
print('Size of the training set: ', train_dataset.get_shape())

# Building the Model

It will take some time to train a model on this dataset. It might be best if you download the pretrained models instead. 

In [0]:
RETRAIN = False

In [0]:
# Create the model.
learning_rate = dc.models.tensorgraph.optimizers.ExponentialDecay(0.01, 0.9, 250)
model = dc.models.TensorGraph(learning_rate=learning_rate, model_dir='models/segmentation')
features = layers.Feature(shape=(None, 520, 696, 1)) / 255.0
labels = layers.Label(shape=(None, 520, 696, 1)) / 255.0
# Downsample three times.
conv1 = layers.Conv2D(16, kernel_size=5, stride=2, in_layers=features)
conv2 = layers.Conv2D(32, kernel_size=5, stride=2, in_layers=conv1)
conv3 = layers.Conv2D(64, kernel_size=5, stride=2, in_layers=conv2)
# Do a 1x1 convolution.
conv4 = layers.Conv2D(64, kernel_size=1, stride=1, in_layers=conv3)
# Upsample three times.
concat1 = layers.Concat(in_layers=[conv3, conv4], axis=3)
deconv1 = layers.Conv2DTranspose(32, kernel_size=5, stride=2, in_layers=concat1)
concat2 = layers.Concat(in_layers=[conv2, deconv1], axis=3)
deconv2 = layers.Conv2DTranspose(16, kernel_size=5, stride=2, in_layers=concat2)
concat3 = layers.Concat(in_layers=[conv1, deconv2], axis=3)
deconv3 = layers.Conv2DTranspose(1, kernel_size=5, stride=2, in_layers=concat3)
# Compute the final output.
concat4 = layers.Concat(in_layers=[features, deconv3], axis=3)
logits = layers.Conv2D(1, kernel_size=5, stride=1, activation_fn=None, in_layers=concat4)
output = layers.Sigmoid(logits)
model.add_output(output)
loss = layers.ReduceSum(layers.SigmoidCrossEntropy(in_layers=(labels, logits)))
model.set_loss(loss)


In [0]:
if not os.path.exists('./models'):
  os.mkdir('models')
if not os.path.exists('./models/segmentation'):
  os.mkdir('models/segmentation')

if not RETRAIN:
  model.restore()

# Displaying the Results

In [0]:
# Train it and evaluate performance on the test set.
if RETRAIN:
  print("About to fit model for 50 epochs")
  model.fit(train_dataset, nb_epoch=50, checkpoint_interval=100)
scores = []
for x, y, w, id in test_dataset.itersamples():
  y_pred = model.predict_on_batch([x]).squeeze()
  scores.append(np.mean((y>0) == (y_pred>0.5)))
print(np.mean(scores))